In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import requests
import re
import io
from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from konlpy.tag import Okt
from urllib.request import HTTPError
from urllib.request import URLError
import time
import json
import csv

In [ ]:
def nate_Crawler():
    
    browser = Chrome()
    browser.maximize_window()
    base_url = 'https://pann.nate.com'
    browser.get(base_url)
    
    #톡커들의 선택 명예의 전당 - 일상톡 랭킹 - 실시간 (당일 실시간 베스트 게시물)
    browser.find_elements_by_xpath('//*[@id="container"]/div[5]/div[2]/div[2]/div[1]/a')[0].click()
    browser.find_elements_by_xpath('//*[@id="tselect"]/a')[0].click()
    browser.find_elements_by_xpath('//*[@id="container"]/div[3]/div[1]/div[2]/div[2]/ul/li[2]/a')[0].click()
    
    res = requests.get(browser.current_url)
    res.raise_for_status()
    res.encoding = None
    html = res.text

    soup = BeautifulSoup(html,'html.parser')

    titles = []
    links = []

    first_list = soup.find('div', {'class': 'cntList'}).findAll('li')

    for li in first_list: 
        f_link = li.findAll('a')
        for a in f_link:
            real_link = 'https://pann.nate.com' + a.get('href')  
        links.append(real_link)

    for li in first_list:
        f_title = li.findAll('dl')
        for dl in f_title:
            t = dl.find('a')
            real_title = t.get('title')
        titles.append(real_title)

    browser.find_elements_by_xpath('//*[@id="last"]')[0].click()

    res = requests.get(browser.current_url)
    res.raise_for_status()
    res.encoding = None
    html = res.text

    soup = BeautifulSoup(html,'html.parser')

    second_list = soup.find('div', {'class': 'cntList'}).findAll('li')

    for li in second_list: 
        s_link = li.findAll('a')
        for a in s_link:
            real_link = 'https://pann.nate.com' + a.get('href')  
        links.append(real_link)

    for li in second_list:
        s_title = li.findAll('dl')
        for dl in s_title:
            t = dl.find('a')
            real_title = t.get('title')
        titles.append(real_title)


    txt = []
    
    for i in links:
        try: 
            #본문
            res = requests.get(i)
            res.raise_for_status()
            res.encoding = None
            html2 = res.text

            soup = BeautifulSoup(html2, 'html.parser')
            contentArea = soup.find("div", {"class" : "viewarea"})
            parags = contentArea.findAll("div", {"id" : "contentArea"})

            content = ""

            for parag in parags:
                content += parag.text.replace('&nbsp;| |\t|\r|\n|', '')
            content = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', content)
            content = re.sub('[&nbsp; | &nbsp;| \n|\t|\r]', '', content)
            content = re.sub('[\xa0]', '', content)
            content = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', content)
            content = re.sub('<[^>]+>','',content)

            txt.append(content)
             
        except HTTPError as e:
            txt.append('')
        except URLError as e:
            txt.append('')
        except AttributeError as e:
            txt.append('')
                      

    okt=Okt()
    title_morphs = []
    txt_morphs = []
    
    for i in titles:
        title_morphs.append(okt.morphs(i))
    
    for i in txt:
        txt_morphs.append(okt.morphs(i))
             
    
    nate_dict = {
        '제목' : titles,
        '본문' : txt
    }

    df = pd.DataFrame(nate_dict) 
    df.to_csv('natepann.csv', index=False, encoding="utf-8-sig")
    
    
    morphs_dict = {
        '제목 형태소' : title_morphs,
        '본문 형태소' : txt_morphs
    }
    df2 = pd.DataFrame(morphs_dict)
    df2.to_csv('natepann_Morphs.csv', index=False, encoding="utf-8-sig")
    
    #browser.close()

In [ ]:
nate_Crawler()